# 40 Build sample data (loto6)
合成データで bronze→silver(long)→gold(features) を作り、Parquet保存。

In [1]:
from fcast.dataset.sample import make_sample_loto, wide_to_long
from fcast.lake.paths import table_path, ensure_dirs
from fcast.lake.io import write_parquet, read_parquet
from fcast.feat.generators import add_basic_transforms
ensure_dirs()


WindowsPath('C:/fcast/notebooks/forecast_lake')

In [2]:
wide = make_sample_loto("loto6", n_draws=120, seed=1)
wide.head()


,ds,N1,N2,N3,N4,N5,N6
0,2020-01-05,22,29,10,24,7,39
1,2020-01-12,24,14,26,34,8,12
2,2020-01-19,34,42,20,40,30,20
3,2020-01-26,42,3,37,21,18,24
4,2020-02-02,3,39,31,32,9,22


In [3]:
# bronze
bronze_path = table_path("bronze","bronze_loto6_draw_raw")
write_parquet(wide, bronze_path)
print(bronze_path)


C:\fcast\notebooks\forecast_lake\bronze\bronze_loto6_draw_raw.parquet


In [4]:
long = wide_to_long(wide, k=6)
long.head()


,unique_id,ds,y
0,N1,2020-01-05,22
1,N1,2020-01-12,24
2,N1,2020-01-19,34
3,N1,2020-01-26,42
4,N1,2020-02-02,3


In [5]:
silver_path = table_path("silver","silver_loto6_long")
write_parquet(long, silver_path)
print(silver_path)


C:\fcast\notebooks\forecast_lake\silver\silver_loto6_long.parquet


In [6]:
gold = add_basic_transforms(long, y_col="y", windows=(3,7))
gold.head()


,unique_id,ds,y,hist_y_diff1,hist_y_cumsum,hist_y_roll_sum_3,hist_y_roll_mean_3,hist_y_roll_sum_7,hist_y_roll_mean_7
0,N1,2020-01-05,22,NaN,22,NaN,NaN,NaN,NaN
1,N1,2020-01-12,24,2.0,46,NaN,NaN,NaN,NaN
2,N1,2020-01-19,34,10.0,80,80.0,26.666667,NaN,NaN
3,N1,2020-01-26,42,8.0,122,100.0,33.333333,NaN,NaN
4,N1,2020-02-02,3,-39.0,125,79.0,26.333333,NaN,NaN


In [7]:
gold_path = table_path("gold","gold_loto6_features")
write_parquet(gold, gold_path)
print(gold_path)


C:\fcast\notebooks\forecast_lake\gold\gold_loto6_features.parquet


In [8]:
# reload and sanity check
df = read_parquet(gold_path)
df.shape, df.columns[:10]


((720, 9),
 Index(['unique_id', 'ds', 'y', 'hist_y_diff1', 'hist_y_cumsum',
        'hist_y_roll_sum_3', 'hist_y_roll_mean_3', 'hist_y_roll_sum_7',
        'hist_y_roll_mean_7'],
       dtype='str'))